Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install pytorch-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install pytorch_pretrained_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 40.3 MB/s 
     |████████████████████████████████| 132 kB 66.4 MB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 9.3 MB 60.9 MB/s 
     |████████████████████████████████| 140 kB 48.6 MB/s 
     |████████████████████████████████| 127 kB 66.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 34.7 MB/s 
     |████████████████████████████████| 163 kB 66.8 MB/s 
     |████████████████████████████████| 7.6 MB 55.4 MB/s 


In [7]:
import sys
import numpy as np
import random as rn
import pandas as pd
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
# from torchnlp.datasets import imdb_dataset      # --> We are using our own uploaded dataset.
from pytorch_pretrained_bert import BertTokenizer
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
from transformers import AutoTokenizer, AutoModelForMaskedLM
import matplotlib.pyplot as plt


import sqlite3
import jieba
import jieba.posseg as pseg

In [8]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

In [9]:
import pathlib
pathlib.Path().resolve()

PosixPath('/content')

In [10]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

In [11]:
train_data = train_data.to_dict(orient='records')
test_data = test_data.to_dict(orient='records')
len(train_data), len(test_data)

(14766, 4922)

In [12]:
train_texts, train_labels = list(zip(*map(lambda d: (d['title'], d['label']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['title'], d['label']), test_data)))
type(train_labels)
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(14766, 14766, 4922, 4922)

In [19]:
def jiebaSlice(content,mode):
    stopword_set = []
    with open('/content/drive/MyDrive/Colab Notebooks/stopword.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.append(stopword.strip('\n'))

    content = content.strip('\n')
    if mode == "POSSEG":
        words = pseg.cut(content,use_paddle=True)
        slicedWords = []
        for word, flag in words:
            if word not in stopword_set:
                slicedWords.append(word)
        return slicedWords
    elif mode == "CUT_HMM":
        seg_list = jieba.cut(content,HMM=True,cut_all=True)
        slicedWords = list(seg_list)
        return slicedWords
    elif mode == "CUT_FOR_SEARCH":
        seg_list = jieba.cut_for_search(content,HMM=True)
        slicedWords = list(seg_list)
        return slicedWords
    elif mode == "NORMAL":
        seg_list = jieba.cut_for_search(content)
        slicedWords = list(seg_list)
        return slicedWords  

In [20]:
jiebaSlice(train_texts[3],'CUT_FOR_SEARCH')

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.828 seconds.
DEBUG:jieba:Loading model cost 0.828 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


['國家', '兩廳', '院發', '這樣', '的', '文', '，', '然後就', '不出', '意料', '的', '出事', '了']

In [21]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [22]:
tokenizer.tokenize('我很開心') #可能要斷詞再tokenize

['我', '很', '開', '心']

In [23]:
jieba.tokenize('我很開心')

<generator object Tokenizer.tokenize at 0x7f44cb241a50>

In [24]:
tokenizer.tokenize(train_texts[10])

['剛', '跟', '初', '戀', '分', '手', '，', '走', '不', '出', '來']

In [25]:
tokenizer.convert_tokens_to_ids

<bound method PreTrainedTokenizerFast.convert_tokens_to_ids of PreTrainedTokenizerFast(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})>

In [29]:
print(train_texts[3])

國家兩廳院發這樣的文，然後就不出意料的出事了


In [30]:
train_tokens = list(map(lambda t: ['[CLS]'] + jiebaSlice(t,'CUT_FOR_SEARCH') + ['[SEP]'], train_texts))

test_tokens = list(map(lambda t: ['[CLS]'] + jiebaSlice(t,'CUT_FOR_SEARCH') + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)

(14766, 4922)

In [31]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

((14766, 512), (4922, 512))

In [32]:
print(train_tokens[30])

['[CLS]', '非常', '律師', 'ep3', '-', '自閉', '症家屬', '的', '理想', '與', '現實', '[SEP]']


In [33]:
print(train_tokens_ids[10])

[ 101  100  100  100 8024 6624  679  100  102    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [34]:
train_y = np.array(train_labels)
test_y = np.array(test_labels) 
# print(train_y)
train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

((14766,), (4922,), 5.501219016659895, 5.5018285249898415)

In [35]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [37]:
baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LogisticRegression()).fit(train_texts, train_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [38]:
baseline_predicted = baseline_model.predict(test_texts)

In [39]:
print(classification_report(test_labels, baseline_predicted))

              precision    recall  f1-score   support

           1       0.79      0.24      0.37       492
           2       0.55      0.16      0.25       492
           3       0.79      0.28      0.41       492
           4       0.58      0.15      0.24       492
           5       0.67      0.22      0.33       492
           6       0.53      0.23      0.32       492
           7       0.12      0.84      0.21       492
           8       0.72      0.28      0.40       492
           9       0.64      0.22      0.33       492
          10       0.79      0.20      0.32       494

    accuracy                           0.28      4922
   macro avg       0.62      0.28      0.32      4922
weighted avg       0.62      0.28      0.32      4922



In [40]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        # model_config = AutoTokenizer.from_pretrained('bert-base-chinese', output_hidden_states=True)
        # self.bert = AutoTokenizer.from_pretrained('bert-base-chinese', config=model_config)
        # self.bert = AutoTokenizer.from_pretrained('bert-base-chinese')
        self.bert = BertModel.from_pretrained('bert-base-chinese')


        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [42]:
bert_clf = BertBinaryClassifier()
bert_clf = bert_clf.cuda()

100%|██████████| 382072689/382072689 [00:36<00:00, 10354761.32B/s]


In [43]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'409.70496M'

In [44]:
x = torch.tensor(train_tokens_ids[:3]).to(device)
y, pooled = bert_clf.bert(x, output_all_encoded_layers=False)
x.shape, y.shape, pooled.shape

(torch.Size([3, 512]), torch.Size([3, 512, 768]), torch.Size([3, 768]))

In [45]:
y = bert_clf(x)
y.cpu().detach().numpy()   

array([[0.27769026],
       [0.30904576],
       [0.28263396]], dtype=float32)

In [46]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'5762.01984M'

In [47]:
y, x, pooled = None, None, None
torch.cuda.empty_cache()     # Clearing Cache space for fresh Model run
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'5762.01984M'

In [48]:
BATCH_SIZE = 4
EPOCHS = 5

In [49]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

'409.70496M'

In [50]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [51]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

In [52]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [53]:
torch.cuda.empty_cache()

In [ ]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)
        
        loss_func = nn.BCELoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

Epoch:  3
1310/3691.5 loss: -446.2242562929062 
1643.589632M


In [ ]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])

In [ ]:
np.mean(bert_predicted)

In [ ]:
print(classification_report(test_y, bert_predicted))